## Prefect

In [ ]:
from prefect import task, Flow
from prefect.tasks.secrets import PrefectSecret
from prefect.tasks.postgres import PostgresExecute, PostgresFetch
import requests
import time
import yaml
import prefect
from prefect import task, Flow, unmapped
from prefect.tasks.secrets import PrefectSecret
from prefect.tasks.postgres import PostgresExecute
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [ ]:
with open("../config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile)
api_key = cfg['riotgamesapi']['key']
username = cfg['postgresql']['user']
password = cfg['postgresql']['password']
database = 'teamfighttactics'

riot_header = {"X-Riot-Token": api_key}